In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers

Короткий пайплайн:
- использовать модель BERTа для создания эмбеддингов для датасета SST (это датасет с предложениями-отзывами о фильмах и оценкой: 1 - отзыв с положительной тональностью, а 0 - отзыв с отрицательной тональностью). Результат этой модели (эмбеддинг предложения) передать на вход следующей модели. Воспользуюсь предобученной моделью BERTa. В качества выхода модели BERT использую вычисления первой позиции - [CLS] токена
- создать классификатор, основанный на логистической регрессии для классификации позитивная или негативная тональность была в обработанном бертом предложении

In [2]:
def get_dataSet():
    df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
    return df

In [3]:
df = get_dataSet()

Скачаю предобученную модель BERT и токенизатор

In [60]:
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [5]:
#токинезация. Разбиваю предложения в нужный для входа в BERT формат
def do_tokenization(df):
    tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    return tokenized

In [6]:
#запомню размер максимального числа токенов в предложениях
max_len = 0
for i in do_tokenization(df).values:
    if len(i) > max_len:
        max_len = len(i)
max_len

67

In [13]:
#добавление пиддинга. После токинезации число токенов от разных предложений могло получиться разным, а матрица нам нужна ровная
def add_padding(tokenized):
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    return padded

In [18]:
#добавление масок к токенам
def att_mask(padded):
    masked = np.where(padded != 0, 1, 0)
    return masked

In [17]:
def prepared_dataSet(df):
    prepared_df = add_padding(do_tokenization(df))
    return prepared_df

In [22]:
#Пропускаю через BERT, чтобы найти last_hidden_states
prepared_df = prepared_dataSet(df)
inputs = torch.LongTensor(prepared_df)  
attention_mask = torch.tensor(att_mask(prepared_df))

with torch.no_grad():
    last_hidden_states = model(inputs, attention_mask=attention_mask)

In [24]:
#Нам нужен только первый токен каждого предложения. Способ, которым BERT выполняет классификацию предложений, заключается в том,
#что он добавляет токен под названием «[CLS]« (для классификации) в начале каждого предложения
X = last_hidden_states[0][:,0,:].numpy()
y = df[1]

Использую логистическую регрессию, как классификатор

In [26]:
#спличу получившийся массив на тестовую и тренировочную выборки
train_X, test_X, train_y, test_y = train_test_split(X, y)

In [30]:
logReg_model = LogisticRegression()
logReg_model.fit(train_X, train_y)
logReg_model.score(test_X, test_y)

C:\Users\Aleksandr\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8473988439306358

In [64]:
#Проверю работу модели на своих предложениях. предложения абсолютно случайные ;-)

good_sentences = ["Sasha Khvorov is the greatest guy I have ever met", "This project is awesome and I will prove it"]
bad_sentences = ["I will be so dissapointed if I fail this test task", "other projects sucks comparing to this one"]
test_sentences = pd.DataFrame(good_sentences + bad_sentences)

In [65]:
#Пропускаю через BERT, чтобы найти last_hidden_states для тестовых предложений
prepared_df_test = prepared_dataSet(test_sentences)
inputs_test = torch.LongTensor(prepared_df_test)  
attention_mask_test = torch.tensor(att_mask(prepared_df_test))

with torch.no_grad():
    last_hidden_states_test = model(inputs_test, attention_mask=attention_mask_test)

In [66]:
#посмотрю на вектор после классификатора:
X2 = last_hidden_states_test[0][:,0,:].numpy()
logReg_model.predict(X2)

array([1, 1, 0, 0], dtype=int64)

Модель правильно отнесла первые два предложения к позитивной тональности, а вторые к негативной!